In [6]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from einops import rearrange
import matplotlib.pyplot as plt
import logging

# Set logging level
logging.basicConfig(level=logging.INFO)

# Import AI Edge Torch libraries - adjusted for actual package structure
from ai_edge_torch._convert import converter
# Let's import other modules as needed and check if they exist
try:
    from ai_edge_torch.generative.layers import normalization
    HAS_NORMALIZATION = True
except ImportError:
    HAS_NORMALIZATION = False
    
try:
    from ai_edge_torch.generative.layers import scaled_dot_product_attention
    HAS_SDPA = True
except ImportError:
    HAS_SDPA = False

# Set paths
WEIGHTS_PATH = "/path/to/model_weights.pth"  # Set this to your model weights path
OUTPUT_PATH = "time_series_transformer.tflite"

# Define core building blocks
class CustomLayerNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-5, enable_hlfb: bool = True):
        super().__init__()
        self.enable_hlfb = enable_hlfb
        # Use AI Edge Torch normalization if available, otherwise use PyTorch
        if HAS_NORMALIZATION:
            self.norm = normalization.LayerNorm(
                dim=dim,
                eps=eps,
                enable_hlfb=enable_hlfb
            )
        else:
            self.norm = nn.LayerNorm(dim, eps=eps)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.norm(x)

class SelfAttentionBlock(nn.Module):
    def __init__(self, embed_dim: int, num_heads: int, dropout: float = 0.0, enable_hlfb: bool = True):
        super().__init__()
        assert embed_dim % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.enable_hlfb = enable_hlfb and HAS_SDPA
        
        self.qkv_projection = nn.Linear(embed_dim, 3 * embed_dim, bias=True)
        self.output_projection = nn.Linear(embed_dim, embed_dim, bias=True)
        self.norm = nn.LayerNorm(embed_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        residual = x
        x = self.norm(x)
        
        B, T, _ = x.shape
        qkv = self.qkv_projection(x)
        
        # Reshape for multi-head attention
        qkv = qkv.reshape(B, T, 3, self.num_heads, self.head_dim)
        qkv = qkv.permute(2, 0, 3, 1, 4)  # [3, B, num_heads, T, head_dim]
        q, k, v = qkv[0], qkv[1], qkv[2]  # Each has shape [B, num_heads, T, head_dim]
        
        # Use scaled_dot_product_attention
        if self.enable_hlfb:
            # For better on-device performance, use the HLFB version if available
            attn = scaled_dot_product_attention.scaled_dot_product_attention_with_hlfb(
                q, k, v, self.head_dim
            )
        else:
            # Standard implementation
            attn = torch.nn.functional.scaled_dot_product_attention(
                q, k, v, dropout_p=0.0, is_causal=False
            )
        
        # Reshape back
        attn = attn.transpose(1, 2).reshape(B, T, -1)
        output = self.output_projection(attn)
        output = self.dropout(output)
        
        return residual + output

class FeedForwardBlock(nn.Module):
    def __init__(self, dim: int, hidden_dim: int, dropout: float = 0.0, 
                 activation: str = "relu"):
        super().__init__()
        # Implement a more standard PyTorch feed-forward network
        if activation == "relu":
            act_fn = nn.ReLU()
        elif activation == "gelu":
            act_fn = nn.GELU()
        else:  # Default to ReLU
            act_fn = nn.ReLU()
            
        self.norm = nn.LayerNorm(dim)
        self.ff = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            act_fn,
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, dim),
            nn.Dropout(dropout)
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x + self.ff(self.norm(x))

# Complete transformer model implementation
class TimeSeriesTransformer(nn.Module):
    def __init__(self,
                 input_frames: int = 128,
                 input_channels: int = 4,
                 num_classes: int = 1,
                 embed_dim: int = 32,
                 num_heads: int = 4,
                 num_layers: int = 2,
                 ff_mult: int = 2,
                 activation: str = "relu",
                 dropout: float = 0.1,
                 norm_first: bool = True,
                 enable_hlfb: bool = True):
        super().__init__()
        
        self.norm_first = norm_first
        
        # Input projection with 1D convolution
        pad = (8 - 1) // 2  # Same padding for kernel=8
        self.input_proj = nn.Sequential(
            nn.Conv1d(input_channels, embed_dim, kernel_size=8, stride=1, padding=pad),
            nn.BatchNorm1d(embed_dim)
        )
        
        # Build transformer layers
        self.encoder_layers = nn.ModuleList()
        for _ in range(num_layers):
            attention_block = SelfAttentionBlock(
                embed_dim=embed_dim,
                num_heads=num_heads,
                dropout=dropout,
                enable_hlfb=enable_hlfb
            )
            
            ff_block = FeedForwardBlock(
                dim=embed_dim,
                hidden_dim=embed_dim * ff_mult,
                dropout=dropout,
                activation=activation
            )
            
            self.encoder_layers.append(nn.ModuleList([attention_block, ff_block]))
        
        # Output layers
        self.final_norm = CustomLayerNorm(embed_dim, enable_hlfb=enable_hlfb)
        self.head = nn.Linear(embed_dim, num_classes)
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # x shape: [batch_size, time_steps, channels]
        # Rearrange for Conv1D
        x = x.transpose(1, 2)  # [batch_size, channels, time_steps]
        x = self.input_proj(x)
        x = x.transpose(1, 2)  # [batch_size, time_steps, embed_dim]
        
        # Process through transformer layers
        for attn, ff in self.encoder_layers:
            x = attn(x)
            x = ff(x)
        
        # Final norm and pooling
        x = self.final_norm(x)
        x = x.mean(dim=1)  # Global temporal pooling
        
        # Classification head
        logits = self.head(x)
        return logits

# Model conversion utilities
def convert_model(model, sample_input, output_path, quantize=True):
    """
    Convert a PyTorch model to TFLite format.
    
    Args:
        model: The PyTorch model to convert
        sample_input: A sample input tensor
        output_path: Path where to save the TFLite model
        quantize: Whether to quantize the model (default: True)
    """
    # Set model to evaluation mode
    model.eval()
    
    # Create signature instance
    if isinstance(sample_input, tuple):
        signature = converter.signature_module.Signature(
            name="serving_default",
            module=model,
            sample_args=sample_input,
            sample_kwargs={}
        )
    else:
        signature = converter.signature_module.Signature(
            name="serving_default",
            module=model,
            sample_args=(sample_input,),
            sample_kwargs={}
        )
    
    # Convert using conversion module
    edge_model = converter.conversion.convert_signatures(
        [signature],
        strict_export=False
    )
    
    # Export the model
    edge_model.export(output_path)
    
    print(f"Model successfully converted and saved to {output_path}")
    return edge_model

# Create and initialize model
def create_model():
    model = TimeSeriesTransformer(
        input_frames=128,
        input_channels=3,
        num_classes=1,
        embed_dim=32,
        num_heads=4,
        num_layers=2,
        ff_mult=2,
        activation="relu",
        dropout=0.1,
        norm_first=True,
        enable_hlfb=True
    )
    
    # Load weights if they exist
    WEIGHTS_PATH='ttfstudentTest2_37.pth'
    if os.path.exists(WEIGHTS_PATH):
        print(f"Loading weights from {WEIGHTS_PATH}")
        weights = torch.load(WEIGHTS_PATH, map_location='cpu')
        model.load_state_dict(weights)
    else:
        print(f"Warning: Weights file not found at {WEIGHTS_PATH}. Using random initialization.")
    
    return model

# Test the model with a random input
def test_model(model):
    model.eval()
    sample_input = torch.randn(1, 128, 4)  # [batch_size, time_steps, channels]
    with torch.no_grad():
        output = model(sample_input)
    print(f"Model output shape: {output.shape}")
    return sample_input, output

# Main execution flow
def main():
    # Create and initialize model
    model = create_model()
    
    # Test the model
    sample_input, _ = test_model(model)
    
    # Convert the model to TFLite
    try:
        edge_model = convert_model(model, sample_input, OUTPUT_PATH, quantize=True)
        print("Model conversion process completed!")
        return edge_model
    except Exception as e:
        print(f"Error during model conversion: {e}")
        print("Trying alternative conversion approach...")
        
        # Alternative approach using direct TorchScript
        try:
            # Try using torch.jit.trace directly
            traced_model = torch.jit.trace(model, sample_input)
            traced_model.save("model.pt")
            print("Model saved as TorchScript model.pt")
            
            print("You'll need to use the TFLite Converter tool to convert from TorchScript to TFLite.")
            print("Example command:")
            print("tflite_convert --saved_model_dir=./model.pt --output_file=model.tflite")
            
        except Exception as e2:
            print(f"Error during TorchScript conversion: {e2}")
            print("Please check the AI Edge Torch installation and documentation.")
        
        return None

# Run the main function
try:
    edge_model = main()
except Exception as e:
    print(f"Error in main execution: {e}")
    print("Please verify that AI Edge Torch is properly installed and compatible with your environment.")


Loading weights from ttfstudentTest2_37.pth
Error in main execution: Error(s) in loading state_dict for TimeSeriesTransformer:
	Missing key(s) in state_dict: "encoder_layers.0.0.qkv_projection.weight", "encoder_layers.0.0.qkv_projection.bias", "encoder_layers.0.0.output_projection.weight", "encoder_layers.0.0.output_projection.bias", "encoder_layers.0.0.norm.weight", "encoder_layers.0.0.norm.bias", "encoder_layers.0.1.norm.weight", "encoder_layers.0.1.norm.bias", "encoder_layers.0.1.ff.0.weight", "encoder_layers.0.1.ff.0.bias", "encoder_layers.0.1.ff.3.weight", "encoder_layers.0.1.ff.3.bias", "encoder_layers.1.0.qkv_projection.weight", "encoder_layers.1.0.qkv_projection.bias", "encoder_layers.1.0.output_projection.weight", "encoder_layers.1.0.output_projection.bias", "encoder_layers.1.0.norm.weight", "encoder_layers.1.0.norm.bias", "encoder_layers.1.1.norm.weight", "encoder_layers.1.1.norm.bias", "encoder_layers.1.1.ff.0.weight", "encoder_layers.1.1.ff.0.bias", "encoder_layers.1.1.ff.3

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from einops import rearrange, reduce

class TransModel(nn.Module):
    """
    Time Series Transformer for fall detection compatible with existing checkpoint
    """
    def __init__(self, 
                 num_layers=2,
                 norm_first=True,
                 embed_dim=32,
                 activation='relu',
                 acc_coords=3,  # Must be 3 to match checkpoint
                 num_classes=1,
                 acc_frames=128,
                 mocap_frames=128,
                 num_heads=4,
                 ff_dim=64,
                 dropout=0.1):
        super().__init__()
        
        # Store configuration
        self.acc_frames = acc_frames
        self.acc_coords = acc_coords
        self.embed_dim = embed_dim
        
        # Input projection - Conv1D + BatchNorm (exact match to checkpoint)
        self.in_proj = nn.Sequential(
            nn.Conv1d(acc_coords, embed_dim, kernel_size=8, stride=1, padding=3),
            nn.BatchNorm1d(embed_dim)
        )
        
        # Build encoder layers (must match checkpoint structure exactly)
        self.encoder = nn.ModuleList()
        for _ in range(num_layers):
            # Each layer has an attention block and a feedforward block
            attn_block = SelfAttentionBlock(
                embed_dim=embed_dim,
                num_heads=num_heads,
                dropout=dropout
            )
            
            ff_block = FeedForwardBlock(
                embed_dim=embed_dim,
                ff_dim=ff_dim,
                dropout=dropout
            )
            
            self.encoder.append(nn.ModuleList([attn_block, ff_block]))
        
        # Final normalization (named to match checkpoint)
        self.temporal_norm = nn.LayerNorm(embed_dim)
        
        # Output head
        self.head = nn.Linear(embed_dim, num_classes)
    
    def forward(self, x):
        """Forward pass for fall detection"""
        # Handle different input formats
        if isinstance(x, dict):
            if 'accelerometer' in x:
                x = x['accelerometer']
            else:
                x = next(iter(x.values()))
        
        # Check input shape and format correctly
        if x.dim() == 3:
            if x.shape[1] == self.acc_coords and x.shape[2] == self.acc_frames:
                # Already in format [B, C, T]
                pass
            elif x.shape[1] == self.acc_frames and x.shape[2] == self.acc_coords:
                # Convert from [B, T, C] to [B, C, T]
                x = x.transpose(1, 2)
            else:
                raise ValueError(f"Expected input shape (B, {self.acc_coords}, {self.acc_frames}) or "
                               f"(B, {self.acc_frames}, {self.acc_coords}), got {x.shape}")
        
        # Replace NaN values with zeros for stability
        x = torch.nan_to_num(x, nan=0.0)
        
        # Apply convolutional projection
        x = self.in_proj(x)  # [B, C, T]
        
        # Transpose to sequence format for transformer
        x = x.transpose(1, 2)  # [B, T, C]
        
        # Apply transformer encoder layers
        for attn, ff in self.encoder:
            x = attn(x)
            x = ff(x)
        
        # Apply final normalization
        x = self.temporal_norm(x)
        
        # Global average pooling over sequence dimension
        x = x.mean(dim=1)
        
        # Apply final classification head
        x = self.head(x)
        
        return x


class SelfAttentionBlock(nn.Module):
    """Self-attention block matching checkpoint structure"""
    def __init__(self, embed_dim, num_heads, dropout=0.1):
        super().__init__()
        self.ln = nn.LayerNorm(embed_dim)
        
        # Single QKV projection as in the checkpoint
        self.qkv = nn.Linear(embed_dim, embed_dim * 3, bias=False)
        
        # Output projection
        self.proj = nn.Linear(embed_dim, embed_dim, bias=False)
        
        # Store parameters
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        self.head_dim = embed_dim // num_heads
        self.scale = self.head_dim ** -0.5
    
    def forward(self, x):
        # Apply residual connection pattern
        residual = x
        
        # Layer normalization
        x = self.ln(x)
        
        # Get shapes
        B, N, C = x.shape
        
        # Project to q, k, v
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, self.head_dim).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]  # [B, H, N, D]
        
        # Calculate attention scores
        attn = (q @ k.transpose(-2, -1)) * self.scale
        
        # Create causal mask (lower triangular)
        mask = torch.triu(torch.ones(N, N, device=x.device), diagonal=1) * -1e9
        attn = attn + mask.unsqueeze(0).unsqueeze(0)
        
        # Apply softmax
        attn = F.softmax(attn, dim=-1)
        
        # Apply attention to values
        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        
        # Apply output projection
        x = self.proj(x)
        
        # Add residual connection
        x = x + residual
        
        return x


class FeedForwardBlock(nn.Module):
    """Feed-forward block matching checkpoint structure"""
    def __init__(self, embed_dim, ff_dim, dropout=0.1):
        super().__init__()
        self.ln = nn.LayerNorm(embed_dim)
        self.ff = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim)
        )
    
    def forward(self, x):
        # Apply residual connection pattern
        residual = x
        
        # Layer normalization
        x = self.ln(x)
        
        # Feed-forward network
        x = self.ff(x)
        
        # Add residual connection
        x = x + residual
        
        return x

In [9]:
# AI Edge Torch: Transformer Model Conversion to TFLite
# Matching exactly the structure of the existing weights

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange
import logging

# Set logging level
logging.basicConfig(level=logging.INFO)

# Import AI Edge Torch libraries
try:
    from ai_edge_torch._convert import converter
    HAS_AI_EDGE_TORCH = True
except ImportError:
    print("Warning: ai_edge_torch not found, conversion to TFLite won't be possible")
    HAS_AI_EDGE_TORCH = False

# Set paths
WEIGHTS_PATH = "ttfstudentTest2_37.pth"  # Set to your model weights path
OUTPUT_PATH = "time_series_transformer.tflite"

# This model class exactly matches the structure in the checkpoint file
class TransModel(nn.Module):
    def __init__(self,
                 acc_frames: int = 128,
                 acc_coords: int = 3,  # Must be 3 to match weights
                 num_classes: int = 1,
                 embed_dim: int = 32,
                 num_heads: int = 4,
                 num_layers: int = 2,
                 ff_mult: int = 2,
                 activation: str = "relu"):
        super().__init__()

        # Input projection with 1D convolution - named exactly like in checkpoint
        pad = (8 - 1) // 2  # 3 for k=8
        self.input_proj = nn.Sequential(
            nn.Conv1d(acc_coords, embed_dim, kernel_size=8, stride=1, padding=pad),
            nn.BatchNorm1d(embed_dim)
        )

        # Encoder layers structure - matches checkpoint exactly
        self.encoder = nn.ModuleList()
        for _ in range(num_layers):
            # Create SelfAttentionBlock with exact name matching
            attn = nn.Module()
            attn.ln = nn.LayerNorm(embed_dim)
            attn.qkv = nn.Linear(embed_dim, 3 * embed_dim, bias=False)
            attn.proj = nn.Linear(embed_dim, embed_dim, bias=False)
            
            # Create FeedForwardBlock with exact name matching
            ff = nn.Module()
            ff.ln = nn.LayerNorm(embed_dim)
            ff.ff = nn.Sequential(
                nn.Linear(embed_dim, embed_dim * ff_mult),
                nn.ReLU() if activation == "relu" else nn.GELU(),
                nn.Linear(embed_dim * ff_mult, embed_dim)
            )
            
            # Add both blocks to encoder as in checkpoint
            layer = nn.Module()
            layer.attn = attn
            layer.ff = ff
            self.encoder.append(layer)
            
        # Normalization layer - named exactly as in checkpoint
        self.temporal_norm = nn.LayerNorm(embed_dim)
        
        # Output layer
        self.head = nn.Linear(embed_dim, num_classes)

    def forward(self, acc_data: torch.Tensor) -> torch.Tensor:
        # Handle different input formats
        if isinstance(acc_data, dict):
            if 'accelerometer' in acc_data:
                acc_data = acc_data['accelerometer']
            else:
                acc_data = next(iter(acc_data.values()))
                
        # Make sure input is in the right format [B, C, T] for Conv1D
        if acc_data.dim() == 3 and acc_data.shape[1] != 3:
            acc_data = acc_data.transpose(1, 2)
        
        # Apply input projection
        x = self.input_proj(acc_data)  # B × C × T
        x = rearrange(x, 'b c t -> b t c')  # B × T × C

        # Forward through transformer blocks
        for layer in self.encoder:
            # Self-attention block
            residual = x
            x_norm = layer.attn.ln(x)
            
            # Get QKV
            B, T, C = x_norm.shape
            head_dim = C // 4  # Hardcoded for 4 heads
            
            qkv = layer.attn.qkv(x_norm)
            qkv = qkv.reshape(B, T, 3, 4, head_dim)
            qkv = qkv.permute(2, 0, 3, 1, 4)  # 3, B, H, T, D
            q, k, v = qkv[0], qkv[1], qkv[2]
            
            # Calculate attention
            attn = F.scaled_dot_product_attention(q, k, v, dropout_p=0.0, is_causal=False)
            attn = attn.transpose(1, 2).reshape(B, T, C)
            attn = layer.attn.proj(attn)
            x = residual + attn
            
            # Feed forward block
            residual = x
            x = layer.ff.ln(x)
            x = layer.ff.ff(x)
            x = residual + x

        # Final norm and pooling
        x = self.temporal_norm(x)
        x = x.mean(dim=1)  # Global temporal pooling
        
        # Output head
        return self.head(x)

# Model conversion utility
def convert_model(model, sample_input, output_path, quantize=True):
    """Convert a PyTorch model to TFLite format."""
    if not HAS_AI_EDGE_TORCH:
        print("AI Edge Torch not available. Model conversion skipped.")
        return None
        
    # Set model to evaluation mode
    model.eval()
    
    try:
        # Create signature instance
        if isinstance(sample_input, tuple):
            signature = converter.signature_module.Signature(
                name="serving_default",
                module=model,
                sample_args=sample_input,
                sample_kwargs={}
            )
        else:
            signature = converter.signature_module.Signature(
                name="serving_default",
                module=model,
                sample_args=(sample_input,),
                sample_kwargs={}
            )
        
        # Convert using conversion module
        edge_model = converter.conversion.convert_signatures(
            [signature],
            strict_export=False
        )
        
        # Export the model
        edge_model.export(output_path)
        
        print(f"Model successfully converted and saved to {output_path}")
        return edge_model
    except Exception as e:
        print(f"Error during model conversion: {e}")
        print("Trying alternative conversion approach...")
        
        # Alternative approach using direct TorchScript
        try:
            # Try using torch.jit.trace directly
            traced_model = torch.jit.trace(model, sample_input)
            traced_model.save("model.pt")
            print("Model saved as TorchScript model.pt")
            
            print("You'll need to use the TFLite Converter tool to convert from TorchScript to TFLite.")
            print("Example command:")
            print("tflite_convert --saved_model_dir=./model.pt --output_file=model.tflite")
            
        except Exception as e2:
            print(f"Error during TorchScript conversion: {e2}")
            
        return None

# Main execution code
def main():
    # Create model with exact structure to match checkpoint
    model = TransModel(
        acc_frames=128,
        acc_coords=3,  # Must be 3 to match weights
        num_classes=1,
        embed_dim=32,
        num_heads=4,
        num_layers=2,
        ff_mult=2,
        activation="relu"
    )
    
    # Load weights
    if os.path.exists(WEIGHTS_PATH):
        print(f"Loading weights from {WEIGHTS_PATH}")
        checkpoint = torch.load(WEIGHTS_PATH, map_location='cpu')
        # Handle potential "model_state_dict" wrapper
        if isinstance(checkpoint, dict) and "model_state_dict" in checkpoint:
            checkpoint = checkpoint["model_state_dict"]
        model.load_state_dict(checkpoint)
    else:
        print(f"Warning: Weights file not found at {WEIGHTS_PATH}")
    
    # Set to evaluation mode
    model.eval()
    
    # Test with random input
    sample_input = torch.randn(1, 3, 128)  # [batch_size, channels, time_steps]
    with torch.no_grad():
        output = model(sample_input)
        print(f"Model output shape: {output.shape}")
    
    # Convert model to TFLite
    converted_model = convert_model(model, sample_input, OUTPUT_PATH)
    
    return model, converted_model

# Run the main function
try:
    model, converted_model = main()
    print("Process completed successfully!")
except Exception as e:
    print(f"Error in main execution: {e}")


Loading weights from ttfstudentTest2_37.pth
Model output shape: torch.Size([1, 1])


INFO:root:Use JAX lowering: aten._native_batch_norm_legit_no_training
INFO:root:Use JAX lowering: aten.permute
INFO:root:Use JAX lowering: aten.view
INFO:root:Use JAX lowering: aten.mm
INFO:root:Use JAX lowering: aten.select
INFO:root:Use JAX lowering: aten.mul.Scalar
INFO:root:Use JAX lowering: aten.expand
INFO:root:Use JAX lowering: aten.clone.default
INFO:root:Use JAX lowering: aten.bmm
INFO:root:Use JAX lowering: aten._softmax
INFO:root:Use JAX lowering: aten.add.Tensor
INFO:root:Use JAX lowering: aten.mul.Tensor
INFO:root:Use JAX lowering: aten.relu
INFO:root:Use JAX lowering: aten.mean
I0000 00:00:1745249525.902734   37805 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4057 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2060 with Max-Q Design, pci bus id: 0000:01:00.0, compute capability: 7.5
INFO:absl:Function `inner` contains input name(s) resource with unsupported characters which will be renamed to xlacallmodule_readvariableop_30_reso

INFO:tensorflow:Assets written to: /tmp/tmp0ciygy00/assets


INFO:tensorflow:Assets written to: /tmp/tmp0ciygy00/assets


Model successfully converted and saved to time_series_transformer.tflite
Process completed successfully!


W0000 00:00:1745249527.072029   37805 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1745249527.072117   37805 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-04-21 10:32:07.073339: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmp0ciygy00
2025-04-21 10:32:07.073980: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-04-21 10:32:07.073991: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmp0ciygy00
I0000 00:00:1745249527.080699   37805 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-04-21 10:32:07.081536: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-04-21 10:32:07.135812: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmp0ciygy00
2025-04-21 10:32:07.144849: I tensorflow/cc/saved_model/loader.cc:471] SavedModel 